### Checking wind and WP generation ###

Looking at the wind anomalies in the model, these do not seem to align with the processed wind power generation.

We first want to establish whether this is correct in the observed data

Before moving on to look at the model data


In [1]:
# Local imports
import os
import sys
import glob
import time
import json
import argparse

# Third-party imports
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd
import iris

# Specific imports
from tqdm import tqdm
from datetime import datetime, timedelta

# Local imports
from plot_dnw_circ import extract_obs_data

/home/users/benhutch/.conda/envs/bens-conda-env2/lib/python3.11/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [2]:
# ----------------------
# Set up the hard-coded variables
# ----------------------

# Set up the directory for the obs df
obs_df_dir = "/home/users/benhutch/unseen_multi_year/dfs/"
metadata_dir = "/gws/nopw/j04/canari/users/benhutch/unseen/saved_arrs/metadata/"

# Set up the fname for the obs df
obs_df_fname = "full_field_obs_tas_wind_UK_1961-2024_DJF_detrended_01-07-2025_dnw.csv"

In [3]:
# --------------------
# Load in the obser df
# --------------------

obs_df = pd.read_csv(os.path.join(obs_df_dir, obs_df_fname))

In [4]:
obs_df.head()

,time,data_tas,data_c,data_sfcWind,data_wp_generation,combined_cfs,effective_dec_year,data_c_dt,data_sfcWind_dt,combined_cfs_dt,total_gen,hdd,cdd,data_c_dt_UK_demand,demand_net_wind
0,1961-12-01,278.569544,5.419544,7.573910,9.074235,0.413943,1961,7.628006,7.886256,0.442651,13.475930,7.871994,0,40.941258,27.465327
1,1961-12-02,275.647926,2.497926,8.598442,13.568653,0.618966,1961,4.706388,8.910788,0.647674,19.717606,10.793612,0,43.125484,23.407879
2,1961-12-03,273.728248,0.578248,6.730659,5.470028,0.249528,1961,2.786711,7.043005,0.278236,8.470544,12.713289,0,44.560652,36.090108
3,1961-12-04,277.405764,4.255764,8.079840,12.264395,0.559469,1961,6.464227,8.392186,0.588177,17.906302,9.035773,0,41.811309,23.905007
4,1961-12-05,276.771560,3.621560,11.786189,17.461881,0.796565,1961,5.830022,12.098535,0.825273,25.124372,9.669978,0,42.285446,17.161074


In [5]:
# --------------
# Subset the obs df to effective dec year 1961
# --------------

obs_df_1961 = obs_df[obs_df['effective_dec_year'] == 1961]

# Print the shape of the subsetted df
print(f"Shape of obs_df_1961: {obs_df_1961.shape}")

Shape of obs_df_1961: (90, 15)


In [7]:
%%time

# -----------------
# Extract the obs data specifically for this winter
# For psl, tas, and wind
# -----------------

psl_arr_1961, psl_dates_list_1961 = extract_obs_data(
    obs_df=obs_df_1961,
    variable='psl',
    region='NA',
    time_freq='day',
    season='DJF',
    lat_shape=90,
    lon_shape=96,
    arrs_dir="/gws/nopw/j04/canari/users/benhutch/unseen/saved_arrs/obs/",
    metadata_dir=metadata_dir,
    lats_path=os.path.join(
    metadata_dir, "HadGEM3-GC31-MM_psl_NA_1960_DJF_day_lats.npy"
    ),
    lons_path=os.path.join(
        metadata_dir, "HadGEM3-GC31-MM_psl_NA_1960_DJF_day_lons.npy"
    ),
)

# Do the same for tas
tas_arr_1961, tas_dates_list_1961 = extract_obs_data(
    obs_df=obs_df_1961,
    variable='tas',
    region='Europe',
    time_freq='day',
    season='DJF',
    lat_shape=63,
    lon_shape=49,
    arrs_dir="/gws/nopw/j04/canari/users/benhutch/unseen/saved_arrs/obs/",
    metadata_dir=metadata_dir,
    lats_path=os.path.join(
        metadata_dir, "HadGEM3-GC31-MM_tas_Europe_1960_DJF_day_lats.npy"
    ),
    lons_path=os.path.join(
        metadata_dir, "HadGEM3-GC31-MM_tas_Europe_1960_DJF_day_lons.npy"
    ),
)

# Do the same for wind
wind_arr_1961, wind_dates_list_1961 = extract_obs_data(
    obs_df=obs_df_1961,
    variable='sfcWind',
    region='Europe',
    time_freq='day',
    season='DJF',
    lat_shape=63,
    lon_shape=49,
    arrs_dir="/gws/nopw/j04/canari/users/benhutch/unseen/saved_arrs/obs/",
    metadata_dir=metadata_dir,
    lats_path=os.path.join(
        metadata_dir, "HadGEM3-GC31-MM_sfcWind_Europe_1960_DJF_day_lats.npy"
    ),
    lons_path=os.path.join(
        metadata_dir, "HadGEM3-GC31-MM_sfcWind_Europe_1960_DJF_day_lons.npy"
    ),
)

Dates:  [cftime.datetime(1961, 12, 1, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 2, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 3, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 4, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 5, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 6, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 7, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 8, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 9, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 10, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 11, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 12, 0, 0, 0, 0, calendar='standard', ha

Loading data: 90it [00:00, 591.20it/s]

Dates:  [cftime.datetime(1961, 12, 1, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 2, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 3, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 4, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 5, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 6, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 7, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 8, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 9, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 10, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 11, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 12, 0, 0, 0, 0, calendar='standard', ha

--------------------------------
Shape of data arr detrended: (65, 63, 49)
--------------------------------


Loading data: 90it [02:01,  1.35s/it]


Dates:  [cftime.datetime(1961, 12, 1, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 2, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 3, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 4, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 5, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 6, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 7, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 8, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 9, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 10, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 11, 0, 0, 0, 0, calendar='standard', has_year_zero=False), cftime.datetime(1961, 12, 12, 0, 0, 0, 0, calendar='standard', ha

Loading data: 90it [00:00, 618.88it/s]

CPU times: user 2min, sys: 620 ms, total: 2min
Wall time: 2min 2s
